In [ ]:
../julia-1.10.0/bin/julia --project=. --thread 32

In [ ]:
using HopfieldDCA; import Flux: gradient
filepath = "../DataAttentionDCA/data/PF00014/PF00014_mgap6.fasta.gz"; H = 32; alg = HopPlmVar(H, filepath);tmp = HopfieldDCA.StgArr(alg); _w = alg.W ./ sum(alg.W);
n_ep = 3; η = 0.5; λ = 0.001;
@time m,n = trainer_J(alg, n_ep, η = η, λ = λ);
import Flux: gradient
KK = rand(alg.N, alg.N, alg.H);
gradient((p1,p2)->get_loss_J_tmp(p1,p2, alg.Z, _w, tmp,KK), alg.K, alg.V)
n_ep = 3; η = 0.5; λ = 0.001;
@time m,n = trainer_J(alg, n_ep, η = η, λ = λ);
@time m,n = trainer_J(alg, n_ep, η = η, λ = λ, 
    savefile="log_J/H$(H)η$(η)λ$(λ)T$(n_ep).txt");
tmp = HopfieldDCA.StgArr(alg); _w = alg.W ./ sum(alg.W);
@time get_loss_J(alg.K, alg.V, alg.Z, _w)
@time get_loss_J_tmp(alg.K, alg.V, alg.Z, _w, tmp)



println("Done")
@time m,n = trainer(alg, n_ep, η = η, λ = λ, 
    savefile="log/H$(H)η$(η)λ$(λ)T$(n_ep).txt");


In [ ]:
using HopfieldDCA
import Flux: gradient
filepath = "../DataAttentionDCA/data/PF00014/PF00014_mgap6.fasta.gz"; H = 32; alg = HopPlmVar(H, filepath);tmp = HopfieldDCA.StgArr(alg); _w = alg.W ./ sum(alg.W);
N = alg.N; H = alg.H; q = alg.q; M = size(alg.Z,2); 
KK = rand(N, N, H); J = rand(N,N,q,q); en = zeros(q,N,M); 
data_en = zeros(N,M); log_z = zeros(N,M); loss = zeros(N);
#@time HopfieldDCA.get_loss_J_ext(alg.K, alg.V, alg.Z, _w, KK, J, en, data_en, log_z, loss)
@time gradient((p1,p2)->HopfieldDCA.get_loss_J_ext(p1,p2, alg.Z, _w, KK, J, en, data_en, log_z, loss),p1,p2)  

In [ ]:
structfile = "../DataAttentionDCA/data/PF00014/PF00014_struct.dat"
s = HopfieldDCA.score(m.K, m.V); PPV_h = HopfieldDCA.compute_PPV(s, structfile);

close("all"); plot(ppv_plm[1], label="Plm"); plot(PPV_h, label="HopPlm");legend();savefig("ciao.png");

